In [1]:
# Simlari Assignment - TFIDF keyword extraction

Task:

Implement keyword extraction to extract other interesting keywords / phrases / concepts, which are as common as possible in all the texts.

In [2]:
from nltk.util import ngrams
from tqdm.notebook import tqdm
from pprint import pprint  # pretty-printer
from collections import defaultdict, Counter

#print(stopwords.words('english'))
import spacy
import re
import re
import string
import nltk

#nlp = spacy.load("en_core_web_lg")
nlp = spacy.load("en_core_web_sm")

from math import log


#nltk.download('stopwords',quiet=True)


# File generators
let us make a file generator `docs` to allow us to operate on one file at a time in memory
since patents documents can be large.

In [3]:
import os
folder = './files/'
def named_docs():
    """ a generator for files in a folder. """ 
    for file_name in os.listdir(folder):
        with open(folder + file_name, "r", encoding='utf-8') as text_file:
            #read file to a string & split
            yield (file_name,text_file.read())

lets also make a test data set with the following ranks:




In [4]:
def named_docs_static():
    """ a generator for some static files - for tests """ 
    fakes = [
        ('d01.txt','Alice likes Mr Red who sat on the river bank'),
        ('d02.txt','Bob hates Jack Black'),
        ('d03.txt','Alice misses Charlie Brown but she hits Bob'),
        ('d04.txt','Bob misses Charlie Brown'),
        ('d05.txt','Alice punched Bob and Charlie Brown black and blue'),
        ('d06.txt','Alice Bob and Charlie robbed the bank'),
    ]
    
    for fake in fakes:
        yield fake        

let's put this generator to use by counting the files

In [5]:
def file_counter(generator):
    file_count=0
    for (file_name,doc) in generator():
        file_count += 1
    return file_count

file_count=file_counter(named_docs_static)
print(f'static file count={file_count}')
file_count=file_counter(named_docs)
print(f'real file count={file_count}')

static file count=6
real file count=100


In [6]:
def spacy_tokenizer_lemmatizer(text,debug = False):
    """
    tokenize the text, 
    drop white space token
    drop stop words contextually
    normalize words to lemma form    
    """
    doc = nlp(text)
    #tokens = tokenizer(text)
    
    lemma_list = []
    for token in doc:
        if re.match('\s+',token.text):
            if debug: 
                print(f'- [\t\t] ws')
            continue 
        if re.match(f'^[{string.punctuation}]+$',token.text):
            if debug: 
                print(f'- [{token.lemma_}] punctuation')
            continue
        if re.match(f'^[0-9]+^',token.text):
            if debug: 
                print('+ {token.lemma_} <- {token.text}')
            lemma_list.append("__number__")             
            continue         

        elif token.is_stop is False:
            lemma_list.append(token.lemma_.lower())
#             if token.text =='learn' :
#                 print("***")
            if token.text != token.lemma_:
                if debug: print(f'+ [{token.lemma_} <- {token.text}]/{token.pos_}')                     
            else:
                if debug: print(f'+ [{token.lemma_}]/{token.pos_}')
            continue
        else:
             if debug: print(f'- [{token.lemma_}]/{token.pos_}  stop word')
             
    return lemma_list

def inspect_spacy(doc_filter=[0]):
    for num, (_,doc) in enumerate(named_docs_static()):
        if num in doc_filter:
            spacy_tokenizer_lemmatizer(doc,True)
            
inspect_spacy()

+ [Alice]/PROPN
+ [like <- likes]/VERB
+ [Mr]/PROPN
+ [Red]/PROPN
- [who]/PRON  stop word
+ [sit <- sat]/VERB
- [on]/ADP  stop word
- [the]/DET  stop word
+ [river]/NOUN
+ [bank]/NOUN


# TF-IDF

It possible to do TDIDF in steps or in one function. 
I like short function but I ended one long function, but keeping the data from each step.

I added tokenizeation using spacy with the above method.
I also preferred lemmatization to stemming. These are less important with embedding and larger corpus
but we only have 100 docs.
And I added n-gram support to process phrases using NLTK.

now let's build a vocabulary, count all the term frequencies per document AKA TF, the the n and document frequencies a.k.a DF

In [7]:
doc_id={}   # numeric repersentation for each file
def scanner(generator,max_ngrams=3):
    """
    scans files and returns:
    doc_ids   - doc to number lookup
    term_ids  - term to number lookup
    term_freq - count of (doc,term)
    doc_freq  - count of docs per term
    """
    doc_ids=defaultdict(int)     # map: file        =>  index (1...N)
    term_ids=defaultdict(int)    # map: token       =>  index (1...M)
    term_freq=defaultdict(int)   # map: (doc,token) => count
    doc_freq=defaultdict(int)    # map: token       => doc_count
    idf = defaultdict(int)       # map: doc_count   => log( N+1 ) - log(doc_count+1) 
    tfidf = defaultdict(int)     # map: term_freq   => term_freq * idf
    # tdidf can inflate for long docs 
    doc_lengh = defaultdict(int) # map: file        =>  len(doc)
    tfidf_norm = defaultdict(int)# map: tdidf       =>  tdidf/len(doc) 
    
    #flags
    use_ngrams=True
    use_spacy=True
    
    term_counter=0
    # once per document
    for doc_counter, (f_name,doc) in enumerate(tqdm(generator())):
        # let's enumerate the docs
        doc_ids[f_name] = doc_counter
        
        # lets tokenize
        doc_tokens = [] # words and phrases 
        #tokenize using spacy
        if use_spacy:
            doc_tokens = spacy_tokenizer_lemmatizer(doc)
        else:
            doc_tokens = doc.lower().split()
          
        doc_ngrams = [] #n-grams
        for ngram_size in range(1,max_ngrams+1):
            for ngram in ngrams(doc_tokens,ngram_size):
                doc_ngrams.append(" ".join(ngram))
        if use_ngrams:
            doc_tokens=doc_ngrams  
        
        # lets keep track of document length so we can normalize
        doc_lengh[doc_counter] = len(doc_tokens)+1
        
        # for each token in the doc
        for token in doc_tokens:
            # let's enumerate new tokens
            if token not in term_ids:
                term_ids[token] = term_counter
                term_counter += 1
                
            # let's build term_frequencies
            ###############################            
            term_freq[doc_counter,term_ids[token]] += 1
            
        # just once per token in the document  
        for token in set(doc_tokens):
            # build document frequencies
            ############################
            doc_freq[term_ids[token]]+=1
            
    # number of docs based on the counter which starts at 0
    N = doc_counter + 1
    # let us calculate the idf
    for k,v in doc_freq.items():
        idf[k] = 1 + log(N) - log(v + 1)
        
    # finaly let us calculate tf-idf and tfidf_norm
    for k,v in term_freq.items():
        tf_doc_id=k[0]
        tf_term  =k[1]
        tfidf[k]     = v * idf[tf_term]
        tfidf_norm[k]= v * idf[tf_term] / doc_lengh[tf_doc_id]
        
    return ({"doc_ids" : doc_ids,        
             "term_ids" : term_ids,
             "doc_freq" : doc_freq,
             "idf" : idf,
             "term_freq" : term_freq,
             "tfidf" : tfidf,
             "tfidf_norm" : tfidf_norm,
             "doc_lengh" : doc_lengh
            })

In [8]:
generators = [
   # named_docs_static,
    named_docs,    
]

# inspect 
def inspect(generator,max_idx=2):
    """
        inspecct the first max_idx results in the ds returned by scanner on a generator

        generator - a named file generator
        max_idx - how many items to slice from the results
    """
    results = scanner(generator) 
    print(generator.__name__,":")
    for name, data_dict in results.items():
        print(f'{name} \t: {list(data_dict.items())[0:max_idx]} ... {list(data_dict.items())[-max_idx:]}')
        
    # since the term are set to term ids we might want to see the terms
    # TODO print ds with terms instead of ids
    
    return results

for generator in generators:    
    results=inspect(generator)


named_docs :
doc_ids 	: [('CN110211379A.txt', 0), ('FAM63678824.txt', 1)] ... [('CN113496457A.txt', 98), ('US20200311599A1.txt', 99)]
term_ids 	: [('public', 0), ('transport', 1)] ... [('send order base', 13574), ('order base partially', 13575)]
doc_freq 	: [(104, 1), (128, 1)] ... [(13511, 1), (13558, 1)]
idf 	: [(104, 4.912023005428146), (128, 4.912023005428146)] ... [(13511, 4.912023005428146), (13558, 4.912023005428146)]
term_freq 	: [((0, 0), 4), ((0, 1), 4)] ... [((99, 13574), 1), ((99, 13575), 1)]
tfidf 	: [((0, 0), 18.026231589279927), ((0, 1), 19.648092021712586)] ... [((99, 13574), 4.912023005428146), ((99, 13575), 4.912023005428146)]
tfidf_norm 	: [((0, 0), 0.07769927409172382), ((0, 1), 0.08469005181772667)] ... [((99, 13574), 0.025450896401182106), ((99, 13575), 0.025450896401182106)]
doc_lengh 	: [(0, 232), (1, 238)] ... [(98, 175), (99, 193)]


we can now:
1. [ ] rank terms and phrases per document.
1. [ ] rank phrases using total tf-idf scores on the corpus
1. [ ] move on to embeddings 

In [9]:
term_ids = results["term_ids"]
id_terms = {v:k for (k,v) in term_ids.items() }

#term_ids
#id_terms

before ranking term we should split them into unigrams, bigram and trigram.etc
this will allow us to pick the top items of each speretly


In [10]:
unigrams=[]
bigrams=[]
trigrams=[]

for term,id in term_ids.items():
    tuple_size = len(term.split())
    if tuple_size == 1:
        unigrams.append(id)
    if tuple_size == 2:
        bigrams.append(id)
    if tuple_size == 3:
        trigrams.append(id)
    if False:
        print(f'id:{id:4},length: {len(term):3},tokens:{len(term.split()):3}, term: {term} ')

ngrams_dict = {"unigrams":unigrams,"bigrams":bigrams,"trigrams":trigrams}

def inspect_ngrams(slice=30):    
    for (k,v) in ngrams_dict.items():
        print(f"{k}: {len(v)}")
        print(f'first {slice} {k} are:')
        print(v[0:slice])

inspect_ngrams()

unigrams: 1267
first 30 unigrams are:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
bigrams: 5297
first 30 bigrams are:
[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
trigrams: 7012
first 30 trigrams are:
[103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132]


In [11]:
term_rank = defaultdict(int)
for k,v in results["tfidf_norm"].items():
    doc=k[0]
    term=k[1]
    term_rank[term] += v

In [12]:
import operator
sorted_term_rank = sorted(term_rank.items(), key=operator.itemgetter(1),reverse=True)

In [13]:
def inspect_top_terms(slice=3):
    #iterate over ngram size
    for n,(ng_name,ng_dict) in enumerate(ngrams_dict.items()):
        counter = 0
        slicer = slice/100*len(ng_dict)/(n+1)
        for (term_id,rank) in sorted_term_rank :
            if counter >= slicer:
                continue
            elif term_id in ng_dict:
                print(f"{ng_name} score: {rank:0.2f} \t term: {id_terms[term_id]}")
                counter +=1
        print()
                
inspect_top_terms()

unigrams score: 1.76 	 term: machine
unigrams score: 1.33 	 term: model
unigrams score: 1.21 	 term: optimization
unigrams score: 1.08 	 term: learning
unigrams score: 1.04 	 term: parameter
unigrams score: 1.02 	 term: method
unigrams score: 0.96 	 term: learn
unigrams score: 0.89 	 term: datum
unigrams score: 0.81 	 term: system
unigrams score: 0.80 	 term: device
unigrams score: 0.72 	 term: base
unigrams score: 0.68 	 term: network
unigrams score: 0.67 	 term: set
unigrams score: 0.66 	 term: value
unigrams score: 0.62 	 term: algorithm
unigrams score: 0.57 	 term: plurality
unigrams score: 0.57 	 term: hyper
unigrams score: 0.56 	 term: optimize
unigrams score: 0.54 	 term: configuration
unigrams score: 0.53 	 term: search
unigrams score: 0.52 	 term: training
unigrams score: 0.48 	 term: optimal
unigrams score: 0.48 	 term: include
unigrams score: 0.47 	 term: obtain
unigrams score: 0.45 	 term: process
unigrams score: 0.44 	 term: operation
unigrams score: 0.44 	 term: state
uni